In [1]:
import tkinter as tk
from tkinter import ttk
import requests
import pandas as pd
from datetime import datetime

# Fetch and prepare odds data
def download_odds(selected_bookmakers):
    url = "https://api.the-odds-api.com/v4/sports/mma_mixed_martial_arts/odds/?apiKey=0a961c52965144f19df17552e65f101e&regions=us&markets=h2h,spreads&oddsFormat=american"
    response = requests.get(url)
    result = response.json()
    
    all_matches = []
    for res in result:
        for books in res['bookmakers']:
            if selected_bookmakers != 'All' and books['title'] not in selected_bookmakers:
                continue
            match = {}
            match['event_time'] = datetime.fromisoformat(res['commence_time'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
            match['bookmaker'] = books['title']
            match['fighter1'] = books['markets'][0]['outcomes'][0]['name']
            match['fighter2'] = books['markets'][0]['outcomes'][1]['name']
            match['odds_f1'] = books['markets'][0]['outcomes'][0]['price']
            match['odds_f2'] = books['markets'][0]['outcomes'][1]['price']
            all_matches.append(match)
            
    df = pd.DataFrame(all_matches)
    
    if selected_bookmakers == 'All':
        df = df.groupby(['event_time', 'fighter1', 'fighter2']).apply(lambda x: x.loc[x['odds_f1'].idxmax()]).reset_index(drop=True)
    
    top_fights = df.sort_values('event_time').drop_duplicates(subset=['event_time', 'fighter1', 'fighter2']).head(20)
    
    return top_fights

# Refresh the treeview
def refresh_table(tree, selected_bookmakers):
    for row in tree.get_children():
        tree.delete(row)
    df = download_odds(selected_bookmakers)
    for index, row in df.iterrows():
        tree.insert('', 'end', values=(row['event_time'], row['fighter1'], row['odds_f1'], row['fighter2'], row['odds_f2'], row['bookmaker']))

# Handle bookmaker selection
def on_bookmaker_select(event):
    selected_bookmakers = bookmaker_var.get()
    refresh_table(tree, selected_bookmakers)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Live Updated UFC Odds")
    
    # Bookmaker dropdown menu
    bookmaker_options = ['All', 'Bovada', 'DraftKings', 'BetRivers', 'BetUS']
    bookmaker_var = tk.StringVar(value='All')
    bookmaker_menu = ttk.Combobox(root, textvariable=bookmaker_var, values=bookmaker_options)
    bookmaker_menu['state'] = 'readonly'
    bookmaker_menu.pack(side=tk.TOP)
    bookmaker_menu.bind("<<ComboboxSelected>>", on_bookmaker_select)
    
    frame = ttk.Frame(root)
    frame.pack(side=tk.TOP, padx=20, pady=20, fill=tk.BOTH, expand=tk.YES)

    scrollbar = ttk.Scrollbar(frame)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    tree = ttk.Treeview(frame, columns=('Event Time', 'Fighter1', 'Odds1', 'Fighter2', 'Odds2', 'Bookmaker'), show='headings', yscrollcommand=scrollbar.set)
    tree.heading('Event Time', text='Event Time')
    tree.heading('Fighter1', text='Fighter 1')
    tree.heading('Odds1', text='Odds 1')
    tree.heading('Fighter2', text='Fighter 2')
    tree.heading('Odds2', text='Odds 2')
    tree.heading('Bookmaker', text='Bookmaker')
    tree.pack(fill=tk.BOTH, expand=tk.YES)

    scrollbar.config(command=tree.yview)
    
    root.after(1000, refresh_table, tree, 'All')
    root.mainloop()
